# Phase 1: Data Exploration - TCGA-LUAD Dataset

## Objectives
- Load and inspect TCGA-LUAD gene expression data
- Understand data structure and characteristics
- Generate summary statistics
- Identify tumor vs normal samples
- Visualize data distribution
- Document findings for Phase 2 preprocessing

## Dataset Information
- **Source**: TCGA-LUAD (The Cancer Genome Atlas - Lung Adenocarcinoma)
- **Platform**: UCSC Xena Browser
- **Data Type**: Gene Expression RNA-seq (STAR – TPM)
- **Format**: Tab-separated values (TSV)
- **Features**: ~20,000 genes (Ensembl IDs)
- **Samples**: ~589 total (tumor + normal)

In [ ]:
# Import libraries
import sys
from pathlib import Path

# Add backend to path
sys.path.append(str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from backend.config import RAW_DATA_DIR, TUMOR_SAMPLE_SUFFIX, NORMAL_SAMPLE_SUFFIX
from backend.utils import set_random_seeds

# Set random seeds
set_random_seeds(42)

# Plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

print("Libraries imported successfully!")
print(f"Data directory: {RAW_DATA_DIR}")

## 1. Load the Dataset

In [ ]:
# File path
data_file = RAW_DATA_DIR / "TCGA-LUAD.star_tpm.tsv"

# Check if file exists
if not data_file.exists():
    print(f"ERROR: Data file not found at {data_file}")
    print("Please run: python scripts/download_data.py")
else:
    print(f"✓ Data file found: {data_file}")
    file_size_mb = data_file.stat().st_size / (1024 * 1024)
    print(f"✓ File size: {file_size_mb:.2f} MB")

In [ ]:
# Load data (this may take a minute)
print("Loading data... (this may take 30-60 seconds)")
df = pd.read_csv(data_file, sep='\t', index_col=0)
print(f"✓ Data loaded successfully!")
print(f"✓ Shape: {df.shape} (genes x samples)")

## 2. Initial Data Inspection

In [ ]:
# Display first few rows
print("First 5 rows, first 5 columns:")
display(df.iloc[:5, :5])

In [ ]:
# Dataset dimensions
n_genes, n_samples = df.shape
print(f"Dataset Dimensions:")
print(f"  - Number of genes: {n_genes:,}")
print(f"  - Number of samples: {n_samples:,}")
print(f"\nGene IDs (index):")
print(f"  - First gene: {df.index[0]}")
print(f"  - Last gene: {df.index[-1]}")
print(f"\nSample IDs (columns):")
print(f"  - First sample: {df.columns[0]}")
print(f"  - Last sample: {df.columns[-1]}")

## 3. Sample Type Analysis (Tumor vs Normal)

In [ ]:
# Extract sample types from TCGA barcodes
# TCGA barcode format: TCGA-XX-XXXX-XXY where Y indicates sample type
# 01A = Primary Solid Tumor
# 11A = Solid Tissue Normal

sample_types = []
for col in df.columns:
    # Extract the sample type code (e.g., "01A" or "11A")
    parts = col.split('-')
    if len(parts) >= 4:
        sample_type_code = parts[3][:3]  # First 3 chars (e.g., "01A")
        if sample_type_code.startswith('01'):
            sample_types.append('Tumor')
        elif sample_type_code.startswith('11'):
            sample_types.append('Normal')
        else:
            sample_types.append('Other')
    else:
        sample_types.append('Unknown')

# Count sample types
sample_type_counts = pd.Series(sample_types).value_counts()
print("Sample Type Distribution:")
print(sample_type_counts)
print(f"\nTumor samples: {sample_type_counts.get('Tumor', 0)}")
print(f"Normal samples: {sample_type_counts.get('Normal', 0)}")

In [ ]:
# Visualize sample type distribution
fig, ax = plt.subplots(figsize=(8, 6))
sample_type_counts.plot(kind='bar', ax=ax, color=['#e74c3c', '#3498db', '#95a5a6'])
ax.set_title('Sample Type Distribution', fontsize=16, fontweight='bold')
ax.set_xlabel('Sample Type', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.tight_layout()
plt.show()

print(f"\n✓ For this project, we will use only TUMOR samples ({sample_type_counts.get('Tumor', 0)} samples)")

## 4. Data Characteristics

In [ ]:
# Missing values
missing_values = df.isnull().sum().sum()
missing_percentage = (missing_values / (df.shape[0] * df.shape[1])) * 100

print("Missing Values Analysis:")
print(f"  - Total missing values: {missing_values:,}")
print(f"  - Percentage: {missing_percentage:.4f}%")

# Check for missing values per gene
genes_with_missing = df.isnull().any(axis=1).sum()
print(f"  - Genes with missing values: {genes_with_missing:,}")

# Check for missing values per sample
samples_with_missing = df.isnull().any(axis=0).sum()
print(f"  - Samples with missing values: {samples_with_missing:,}")

In [ ]:
# Data statistics
print("\nGene Expression Statistics (all samples):")
print(df.describe())

In [ ]:
# Check data range
print("Data Range Analysis:")
print(f"  - Minimum value: {df.min().min():.4f}")
print(f"  - Maximum value: {df.max().max():.4f}")
print(f"  - Mean value: {df.mean().mean():.4f}")
print(f"  - Median value: {df.median().median():.4f}")

# Check if data is log-transformed
print("\n✓ Note: Data appears to be log-transformed (values in reasonable range).")
print("✓ We should NOT apply additional log transformation.")

## 5. Distribution Visualization

In [ ]:
# Sample a random gene for visualization
random_gene = df.sample(1, random_state=42).index[0]
gene_values = df.loc[random_gene]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(gene_values, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title(f'Distribution of {random_gene}', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Expression Value (log-scale)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].axvline(gene_values.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0].axvline(gene_values.median(), color='green', linestyle='--', linewidth=2, label='Median')
axes[0].legend()

# Box plot
axes[1].boxplot(gene_values, vert=True)
axes[1].set_title(f'Box Plot of {random_gene}', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Expression Value (log-scale)', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Overall distribution across all genes and samples
# Sample 10,000 random values for visualization (to avoid memory issues)
sampled_values = df.values.flatten()
sampled_values = np.random.choice(sampled_values, size=min(10000, len(sampled_values)), replace=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(sampled_values, bins=100, color='steelblue', edgecolor='black', alpha=0.7)
ax.set_title('Overall Gene Expression Distribution (10,000 random samples)', fontsize=14, fontweight='bold')
ax.set_xlabel('Expression Value (log-scale)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.axvline(sampled_values.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
ax.axvline(np.median(sampled_values), color='green', linestyle='--', linewidth=2, label='Median')
ax.legend()
plt.tight_layout()
plt.show()

## 6. Gene Variance Analysis

In [ ]:
# Calculate variance for each gene across all samples
gene_variances = df.var(axis=1)

print("Gene Variance Statistics:")
print(f"  - Mean variance: {gene_variances.mean():.4f}")
print(f"  - Median variance: {gene_variances.median():.4f}")
print(f"  - Min variance: {gene_variances.min():.4f}")
print(f"  - Max variance: {gene_variances.max():.4f}")
print(f"\n  - Genes with variance = 0: {(gene_variances == 0).sum()}")
print(f"  - Genes with variance > 0: {(gene_variances > 0).sum()}")

In [ ]:
# Visualize variance distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of variances
axes[0].hist(gene_variances, bins=100, color='coral', edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Gene Variances', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Variance', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_yscale('log')  # Log scale for better visualization

# Cumulative distribution
sorted_variances = np.sort(gene_variances)[::-1]
cumsum_variances = np.cumsum(sorted_variances)
cumsum_percentage = (cumsum_variances / cumsum_variances[-1]) * 100

axes[1].plot(range(len(cumsum_percentage)), cumsum_percentage, color='darkgreen', linewidth=2)
axes[1].axhline(y=80, color='red', linestyle='--', label='80% variance')
axes[1].axhline(y=90, color='orange', linestyle='--', label='90% variance')
axes[1].set_title('Cumulative Variance Explained', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Number of Genes (sorted by variance)', fontsize=12)
axes[1].set_ylabel('Cumulative Variance Explained (%)', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Top variable genes
top_n = 20
top_variable_genes = gene_variances.nlargest(top_n)
print(f"\nTop {top_n} Most Variable Genes:")
for i, (gene, var) in enumerate(top_variable_genes.items(), 1):
    print(f"  {i:2d}. {gene:25s} - Variance: {var:.4f}")

## 7. Sample Correlation Analysis

In [ ]:
# Sample a subset of samples for correlation visualization (to avoid memory issues)
n_samples_to_visualize = 50
sample_subset = df.sample(n_samples_to_visualize, axis=1, random_state=42)

# Compute correlation matrix
print(f"Computing correlation matrix for {n_samples_to_visualize} samples...")
corr_matrix = sample_subset.corr()

# Visualize
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, square=True, 
            linewidths=0, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title(f'Sample Correlation Matrix ({n_samples_to_visualize} samples)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n✓ High correlation indicates samples are similar")
print(f"✓ This is expected for samples from the same tissue type")

## 8. Summary Statistics for Tumor Samples Only

In [ ]:
# Filter tumor samples
tumor_columns = [col for col, stype in zip(df.columns, sample_types) if stype == 'Tumor']
df_tumor = df[tumor_columns]

print(f"Tumor Sample Dataset:")
print(f"  - Shape: {df_tumor.shape} (genes x tumor samples)")
print(f"  - Number of genes: {df_tumor.shape[0]:,}")
print(f"  - Number of tumor samples: {df_tumor.shape[1]:,}")
print(f"\nThis is the dataset we'll use for training!")

In [ ]:
# Tumor-only statistics
print("\nTumor Samples - Expression Statistics:")
print(df_tumor.describe())

## 9. Key Findings Summary

In [ ]:
print("="*70)
print("PHASE 1 DATA EXPLORATION - KEY FINDINGS")
print("="*70)

print(f"\n1. DATASET DIMENSIONS:")
print(f"   - Total samples: {n_samples:,}")
print(f"   - Tumor samples: {df_tumor.shape[1]:,}")
print(f"   - Normal samples: {sample_type_counts.get('Normal', 0)}")
print(f"   - Total genes: {n_genes:,}")

print(f"\n2. DATA QUALITY:")
print(f"   - Missing values: {missing_percentage:.4f}%")
print(f"   - Data is already log-transformed (DO NOT log again)")
print(f"   - Gene IDs have Ensembl version suffixes (need to remove)")

print(f"\n3. GENE VARIANCE:")
print(f"   - Genes with zero variance: {(gene_variances == 0).sum()}")
print(f"   - Mean variance: {gene_variances.mean():.4f}")
print(f"   - Will select top 1,000-2,000 variable genes for modeling")

print(f"\n4. NEXT STEPS (Phase 2):")
print(f"   - Filter for tumor samples only")
print(f"   - Remove Ensembl version suffixes from gene IDs")
print(f"   - Feature selection: keep top 1,000-2,000 variable genes")
print(f"   - Optional: PCA dimensionality reduction (300-500 components)")
print(f"   - Save processed data for GAN training")

print("\n" + "="*70)
print("Phase 1 Complete! Ready for Phase 2: Data Preprocessing")
print("="*70)

## 10. Save Exploration Report

In [ ]:
# Create exploration report
report = {
    "dataset": "TCGA-LUAD",
    "total_samples": int(n_samples),
    "tumor_samples": int(df_tumor.shape[1]),
    "normal_samples": int(sample_type_counts.get('Normal', 0)),
    "total_genes": int(n_genes),
    "missing_values_percentage": float(missing_percentage),
    "data_range": {
        "min": float(df.min().min()),
        "max": float(df.max().max()),
        "mean": float(df.mean().mean()),
        "median": float(df.median().median())
    },
    "variance_stats": {
        "mean_variance": float(gene_variances.mean()),
        "median_variance": float(gene_variances.median()),
        "genes_with_zero_variance": int((gene_variances == 0).sum())
    },
    "notes": [
        "Data is already log-transformed",
        "Gene IDs have Ensembl version suffixes",
        "Will use only tumor samples for clustering",
        "Feature selection needed (top 1000-2000 variable genes)"
    ]
}

# Save report
from backend.utils import save_json
report_path = Path.cwd().parent / "results" / "data_exploration_report.json"
save_json(report, report_path)

print(f"✓ Exploration report saved to: {report_path}")